In [ ]:
'''
Step 1 - Download ollama from https://ollama.com/download 

Step 2 - Run the following command on the terminal : ollama pull tinyllama 

Step 3 - then use this boiler code to test if the tinyllama 
model is successfully downloaded or not

import ollama
def test_tinyllama():
    try:
        print("Testing TinyLlama...")
        response = ollama.generate(
            model='tinyllama',
            prompt='Hello, what can you do?'
        )
        print("\nResponse from TinyLlama:")
        print(response['response'])
    except Exception as e:
        print("Error:", e)

if __name__ == "__main__":
    test_tinyllama()
'''

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import time
import re
from typing import Dict, List, Tuple
import ollama
from openai import AzureOpenAI
from datetime import datetime
import string
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("API_KEY")
azure_endpoint = os.getenv("AZURE_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("API_VERSION")

In [30]:
# Cell 2: ModelEvaluator Class Definition
class ModelEvaluator:
    def __init__(self, azure_endpoint: str, deployment_name: str, api_version: str, api_key: str):
        """Initialize the evaluator with Azure OpenAI credentials"""
        print("\n🔄 Initializing Model Evaluator...")
        try:
            self.azure_client = AzureOpenAI(
                azure_endpoint=azure_endpoint,
                api_key=api_key,
                api_version=api_version
            )
            self.deployment_name = deployment_name
            print("✅ Azure OpenAI client initialized successfully")
            
            ollama.list()
            print("✅ Ollama connection verified")
            
        except Exception as e:
            print(f"❌ Initialization Error: {e}")
            raise

    def create_prompt(self, question: str, choices: List[str]) -> str:
        """Create enhanced prompt with strict answer requirements"""
        choices_text = "\n".join(f"{chr(65 + i)}. {choice}" for i, choice in enumerate(choices))
        
        prompt = f"""Solve this mathematics question step by step and you MUST end with a clear answer selection.

    QUESTION:
    {question}

    CHOICES:
    {choices_text}

    REQUIREMENTS:
    1. Analyze the question carefully
    2. Show ALL mathematical steps clearly
    3. Calculate values precisely
    4. Evaluate each option systematically
    5. You MUST end with EXACTLY ONE of these phrases:
    - "FINAL ANSWER: A" 
    - "FINAL ANSWER: B"
    - "FINAL ANSWER: C"
    - "FINAL ANSWER: D"

    Your complete solution:"""
        return prompt

    def get_gpt4_response(self, prompt: str, question_num: int) -> Tuple[str, float]:
        """Get response from Azure GPT-4"""
        print(f"\n🤖 GPT-4 [Question {question_num}]")
        start_time = time.time()
        try:
            response = self.azure_client.chat.completions.create(
                model=self.deployment_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=2000
            )
            response_text = response.choices[0].message.content
            time_taken = time.time() - start_time
            print(f"⏱️ Time: {time_taken:.2f}s")
            return response_text, time_taken
        except Exception as e:
            print(f"❌ GPT-4 Error: {e}")
            return "", 0

    def get_tinyllama_response(self, prompt: str, question_num: int) -> Tuple[str, float]:
        """Optimized TinyLlama response generation"""
        print(f"\n🦙 TinyLlama [Question {question_num}]")
        start_time = time.time()
        try:
            response = ollama.generate(
                model='tinyllama',
                prompt=prompt,
                options={
                    'num_predict': 1000,  # Reduced from 2000
                    'top_k': 20,          # Reduced from 40
                    'top_p': 0.9,         # Slightly more focused sampling
                    'repeat_penalty': 1.1, # Prevent repetition
                    'temperature': 0.7,    # Add if supported in your Ollama version
                    'stop': ['Question:', 'QUESTION:', '\n\n']  # Stop tokens to prevent rambling
                }
            )
            response_text = response['response']
            time_taken = time.time() - start_time
            print(f"⏱️ Time: {time_taken:.2f}s")
            return response_text, time_taken
        except Exception as e:
            print(f"❌ TinyLlama Error: {str(e)}")
            return "", 0

    def evaluate_mathematical_reasoning(self, response: str) -> float:
        """Enhanced evaluation of mathematical reasoning quality"""
        score = 0.0
        
        # Check for sophisticated mathematical expressions
        math_patterns = {
            'complex_equations': r'[a-z\d]+\s*=\s*[a-z\d\s\+\-\*/\(\)]+',
            'advanced_operations': r'(?:sqrt|log|exp|sin|cos|tan)',
            'fractions': r'\d+/\d+',
            'exponents': r'\d+\^\d+|\d+\*\*\d+',
            'inequalities': r'[<>≤≥]',
            'multi_step_calc': r'\d+\s*[\+\-\*/]\s*\d+\s*=\s*\d+\s*[\+\-\*/]\s*\d+'
        }
        
        # Weight patterns differently
        pattern_weights = {
            'complex_equations': 0.3,
            'advanced_operations': 0.2,
            'fractions': 0.15,
            'exponents': 0.15,
            'inequalities': 0.1,
            'multi_step_calc': 0.3
        }
        
        for pattern_name, pattern in math_patterns.items():
            matches = len(re.findall(pattern, response))
            score += min(pattern_weights[pattern_name], matches * pattern_weights[pattern_name] / 2)
        
        # Check for mathematical vocabulary
        advanced_math_terms = [
            'theorem', 'proof', 'derivative', 'integral', 'function',
            'coefficient', 'polynomial', 'equation', 'algorithm', 'formula',
            'solve', 'calculate', 'simplify', 'factor', 'distribute'
        ]
        
        term_count = sum(1 for term in advanced_math_terms if term in response.lower())
        score += min(0.5, term_count * 0.1)
        
        return min(1.0, score)

    def evaluate_solution_completeness(self, response: str) -> float:
        """Enhanced evaluation of solution completeness"""
        score = 0.0
        
        # Check for comprehensive solution components
        components = {
            'problem_analysis': r'(?:given|we need to find|problem requires|let\'s analyze)',
            'strategy_explanation': r'(?:approach|strategy|method|we can solve|let\'s solve)',
            'detailed_steps': r'(?:step [1-9]|first|second|third|finally)',
            'calculations': r'\d+\s*[\+\-\*/]\s*\d+\s*=',
            'verification': r'(?:verify|check|confirm|therefore)',
            'options_analysis': r'(?:option [ABCD]|analyzing options|compare choices)',
            'final_conclusion': r'(?:final answer|conclusion|therefore.*answer)'
        }
        
        component_weights = {
            'problem_analysis': 0.15,
            'strategy_explanation': 0.15,
            'detailed_steps': 0.2,
            'calculations': 0.2,
            'verification': 0.1,
            'options_analysis': 0.1,
            'final_conclusion': 0.1
        }
        
        for component, pattern in components.items():
            matches = len(re.findall(pattern, response.lower()))
            score += min(component_weights[component], matches * component_weights[component] / 2)
        
        # Penalize very short responses
        if len(response.split()) < 50:
            score *= 0.5
        
        return min(1.0, score)

    def evaluate_explanation_quality(self, response: str) -> float:
        """Enhanced evaluation of explanation quality"""
        score = 0.0
        
        # Check for sophisticated explanation elements
        explanation_elements = {
            'logical_reasoning': r'(?:because|since|as|therefore|thus|hence)',
            'clarity_markers': r'(?:note that|observe|consider|let\'s|we can)',
            'comparative_analysis': r'(?:comparing|in contrast|while|whereas|on the other hand)',
            'methodical_approach': r'(?:following|using|applying|based on|according to)',
            'critical_thinking': r'(?:however|although|despite|nonetheless|furthermore)',
            'option_evaluation': r'(?:option [ABCD]|choice [ABCD]|analyzing.*options)',
            'conclusive_statement': r'(?:final answer|in conclusion|therefore.*answer)'
        }
        
        element_weights = {
            'logical_reasoning': 0.2,
            'clarity_markers': 0.15,
            'comparative_analysis': 0.15,
            'methodical_approach': 0.15,
            'critical_thinking': 0.15,
            'option_evaluation': 0.1,
            'conclusive_statement': 0.1
        }
        
        for element, pattern in explanation_elements.items():
            matches = len(re.findall(pattern, response.lower()))
            score += min(element_weights[element], matches * element_weights[element] / 2)
        
        # Bonus for well-structured paragraphs
        paragraphs = response.split('\n\n')
        if len(paragraphs) >= 3:
            score += 0.1
            
        return min(1.0, score)

    def evaluate_coherence(self, response: str) -> float:
        """Enhanced evaluation of response coherence"""
        if not response:
            return 0.0
            
        score = 0.0
        
        # Check for sophisticated discourse markers
        discourse_markers = {
            'sequence': r'(?:first|second|third|finally|then|next|lastly)',
            'causation': r'(?:therefore|thus|hence|consequently|because|since)',
            'contrast': r'(?:however|although|despite|nonetheless|while)',
            'addition': r'(?:furthermore|moreover|additionally|in addition)',
            'exemplification': r'(?:for example|for instance|such as|specifically)',
            'conclusion': r'(?:in conclusion|to sum up|finally|therefore)'
        }
        
        marker_weights = {
            'sequence': 0.2,
            'causation': 0.2,
            'contrast': 0.15,
            'addition': 0.15,
            'exemplification': 0.15,
            'conclusion': 0.15
        }
        
        for marker_type, pattern in discourse_markers.items():
            matches = len(re.findall(pattern, response.lower()))
            score += min(marker_weights[marker_type], matches * marker_weights[marker_type] / 2)
        
        # Check for mathematical coherence
        if re.search(r'\d+\s*=.*\d+.*=.*\d+', response):
            score += 0.2
            
        # Check for logical flow in option analysis
        if all(re.search(f'option {letter}', response.lower()) for letter in 'ABCD'):
            score += 0.2
            
        return min(1.0, score)

    def calculate_time_efficiency_score(self, time_taken: float) -> float:
        """Enhanced time efficiency scoring"""
        # Adjusted optimal time ranges
        if time_taken <= 3:
            return 1.0
        elif time_taken <= 10:
            return 0.9
        elif time_taken <= 20:
            return 0.7
        elif time_taken <= 30:
            return 0.5
        elif time_taken <= 45:
            return 0.3
        else:
            return 0.1

    def extract_answer(self, response: str) -> str:
        """Improved answer extraction with multiple fallback methods"""
        if not response:
            return ""
        
        # First priority: Look for the exact format
        final_answer_match = re.search(r'FINAL ANSWER:\s*([ABCD])', response, re.IGNORECASE)
        if final_answer_match:
            return final_answer_match.group(1).upper()
        
        # Second priority: Look for "The answer is X" pattern
        answer_is_match = re.search(r'(?:the\s+)?answer\s+is\s+([ABCD])', response, re.IGNORECASE)
        if answer_is_match:
            return answer_is_match.group(1).upper()
        
        # Third priority: Look for option analysis conclusions
        conclusion_patterns = [
            r'(?:therefore|thus|hence|so),?\s+(?:the\s+)?(?:answer\s+)?(?:is|must\s+be|should\s+be|would\s+be)?\s*([ABCD])',
            r'option\s+([ABCD])(?:\s+is|\s+appears\s+to\s+be|\s+seems\s+to\s+be)?\s+(?:the\s+)?correct',
            r'([ABCD])(?:\s+is|\s+appears\s+to\s+be|\s+seems\s+to\s+be)\s+(?:the\s+)?correct',
            r'(?:selecting|choosing|we\s+choose|we\s+select)\s+(?:option\s+)?([ABCD])',
            r'(?:final|correct|right)\s+(?:answer|option|choice)\s+(?:is|:)\s+([ABCD])'
        ]
        
        for pattern in conclusion_patterns:
            match = re.search(pattern, response, re.IGNORECASE)
            if match:
                return match.group(1).upper()
        
        # Fourth priority: Look for emphasized single letter
        emphasized_letter = re.search(r'(?:^|\s|[^\w])([ABCD])(?:\s|$|[^\w])', response)
        if emphasized_letter:
            return emphasized_letter.group(1).upper()
        
        # If all else fails, analyze which option is discussed most positively
        options_analysis = {
            'A': 0, 'B': 0, 'C': 0, 'D': 0
        }
        
        positive_patterns = [
            r'correct', r'right', r'true', r'valid', r'best',
            r'this works', r'this is', r'matches', r'agrees'
        ]
        
        for option in options_analysis:
            context_pattern = f"option {option}.*?(?=option|$)"
            contexts = re.finditer(context_pattern, response, re.IGNORECASE | re.DOTALL)
            
            for context in contexts:
                context_text = context.group(0)
                for positive_pattern in positive_patterns:
                    if re.search(positive_pattern, context_text, re.IGNORECASE):
                        options_analysis[option] += 1
        
        if any(options_analysis.values()):
            return max(options_analysis.items(), key=lambda x: x[1])[0]
        
        return "A"  # Default fallback

    def evaluate_response(self, response: str, correct_answer: str, time_taken: float) -> Dict:
        """Enhanced response evaluation with adjusted weights"""
        if not response:
            return {
                'correctness': 0,
                'mathematical_reasoning': 0,
                'solution_completeness': 0,
                'explanation_quality': 0,
                'coherence': 0,
                'time_efficiency': 0
            }

        extracted_answer = self.extract_answer(response)
        
        # Enhanced scoring system
        evaluation = {
            'correctness': 1.0 if extracted_answer == correct_answer else 0.0,
            'mathematical_reasoning': self.evaluate_mathematical_reasoning(response),
            'solution_completeness': self.evaluate_solution_completeness(response),
            'explanation_quality': self.evaluate_explanation_quality(response),
            'coherence': self.evaluate_coherence(response),
            'time_efficiency': self.calculate_time_efficiency_score(time_taken)
        }
        
        return evaluation

    def calculate_final_score(self, eval_dict: Dict) -> float:
        """Balanced scoring with emphasis on correctness"""
        weights = {
            'correctness': 0.40,        # Increased
            'mathematical_reasoning': 0.25,
            'solution_completeness': 0.15,
            'explanation_quality': 0.10,
            'coherence': 0.05,
            'time_efficiency': 0.05
        }
        
        base_score = sum(eval_dict[key] * weights[key] for key in eval_dict)
        
        # Bonus for correct answers with good reasoning
        if eval_dict['correctness'] == 1.0 and eval_dict['mathematical_reasoning'] > 0.7:
            base_score *= 1.1
        
        return min(1.0, base_score)

    def evaluate_questions(self, input_csv: str, output_csv: str):
        """Main evaluation function - 15 questions per subject"""
        print("\n📚 Loading questions from CSV...")
        df = pd.read_csv(input_csv)
        
        # Sample 15 questions from each subject
        sampled_df = df.groupby('Subject', group_keys=False).apply(lambda x: x.sample(n=min(len(x), 15)))
        print(f"✅ Loaded {len(sampled_df)} questions for evaluation")
        print("\nDistribution of questions by subject:")
        print(sampled_df['Subject'].value_counts())
        
        results = []
        start_time = time.time()
        total_questions = len(sampled_df)
        
        for idx, row in sampled_df.iterrows():
            question_num = idx + 1
            print(f"\n{'='*60}")
            print(f"📝 Question {len(results)+1}/{total_questions}")
            print(f"Subject: {row['Subject']}")
            print(f"Question: {row['Question']}")
            
            # Create prompt
            prompt = self.create_prompt(row['Question'], eval(row['Choices']))
            
            # Get model responses
            gpt4_response, gpt4_time = self.get_gpt4_response(prompt, len(results)+1)
            tinyllama_response, tinyllama_time = self.get_tinyllama_response(prompt, len(results)+1)
            
            # Evaluate responses
            print("\n📊 Evaluating responses...")
            gpt4_eval = self.evaluate_response(gpt4_response, row['Answer'], gpt4_time)
            tinyllama_eval = self.evaluate_response(tinyllama_response, row['Answer'], tinyllama_time)
            
            # Calculate scores
            gpt4_score = self.calculate_final_score(gpt4_eval)
            tinyllama_score = self.calculate_final_score(tinyllama_eval)
            
            # Print detailed scores
            print(f"\nDetailed Scores for Question {len(results)+1}:")
            print("\nGPT-4:")
            for metric, score in gpt4_eval.items():
                print(f"{metric}: {score:.2f}")
            print(f"Final Score: {gpt4_score:.2f}")
            
            print("\nTinyLlama:")
            for metric, score in tinyllama_eval.items():
                print(f"{metric}: {score:.2f}")
            print(f"Final Score: {tinyllama_score:.2f}")
            
            # Store results
            result = {
                'Question_Number': len(results)+1,
                'Question': row['Question'],
                'Subject': row['Subject'],
                'Correct_Answer': row['Answer'],
                'GPT4_Response': gpt4_response,
                'GPT4_Time': gpt4_time,
                **{f'GPT4_{k}': v for k, v in gpt4_eval.items()},
                'GPT4_Final_Score': gpt4_score,
                'TinyLlama_Response': tinyllama_response,
                'TinyLlama_Time': tinyllama_time,
                **{f'TinyLlama_{k}': v for k, v in tinyllama_eval.items()},
                'TinyLlama_Final_Score': tinyllama_score
            }
            results.append(result)
            
            # Save intermediate results every 5 questions
            if len(results) % 5 == 0:
                print(f"\n💾 Saving intermediate results...")
                pd.DataFrame(results).to_csv(output_csv, index=False)
                
            # Progress indicator
            print(f"\nProgress: {len(results)}/{total_questions} questions completed ({(len(results)/total_questions)*100:.1f}%)")
        
        # Save final results
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_csv, index=False)
        
        # Calculate and display final statistics
        total_time = time.time() - start_time
        print("\n📊 Final Statistics:")
        print(f"Total questions processed: {len(results)}")
        print(f"Total time taken: {total_time/60:.2f} minutes")
        print(f"Average time per question: {total_time/len(results):.2f} seconds")
        
        # Print subject-wise performance
        print("\n📊 Subject-wise Performance:")
        for subject in results_df['Subject'].unique():
            subject_df = results_df[results_df['Subject'] == subject]
            print(f"\n{subject}:")
            print(f"GPT-4 Average Score: {subject_df['GPT4_Final_Score'].mean():.2f}")
            print(f"TinyLlama Average Score: {subject_df['TinyLlama_Final_Score'].mean():.2f}")
        
        print("\nOverall Average Scores by Metric:")
        metrics = ['correctness', 'mathematical_reasoning', 'solution_completeness', 
                'explanation_quality', 'coherence', 'time_efficiency']
        
        print("\nGPT-4:")
        for metric in metrics:
            avg_score = results_df[f'GPT4_{metric}'].mean()
            print(f"{metric}: {avg_score:.2f}")
            
        print("\nTinyLlama:")
        for metric in metrics:
            avg_score = results_df[f'TinyLlama_{metric}'].mean()
            print(f"{metric}: {avg_score:.2f}")
            
        print(f"\nFinal Overall Average Scores:")
        print(f"GPT-4: {results_df['GPT4_Final_Score'].mean():.2f}")
        print(f"TinyLlama: {results_df['TinyLlama_Final_Score'].mean():.2f}")
        
        # Generate performance comparison visualizations
        self.generate_performance_visualizations(results_df, output_csv.replace('.csv', '_analysis.html'))
        
        print(f"\n✅ Evaluation completed - Results saved to {output_csv}")

    def generate_performance_visualizations(self, results_df: pd.DataFrame, output_file: str):
        """Generate detailed performance comparison visualizations"""
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        # Create a detailed HTML report
        html_content = ['<html><head><style>',
                       'body { font-family: Arial, sans-serif; margin: 20px; }',
                       'table { border-collapse: collapse; width: 100%; }',
                       'th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }',
                       'th { background-color: #f2f2f2; }',
                       '</style></head><body>']
        
        # Add summary statistics
        html_content.append('<h2>Model Performance Comparison</h2>')
        
        # Create summary table
        metrics = ['correctness', 'mathematical_reasoning', 'solution_completeness', 
                  'explanation_quality', 'coherence', 'time_efficiency']
        
        html_content.append('<table>')
        html_content.append('<tr><th>Metric</th><th>GPT-4</th><th>TinyLlama</th><th>Difference</th></tr>')
        
        for metric in metrics:
            gpt4_score = results_df[f'GPT4_{metric}'].mean()
            tiny_score = results_df[f'TinyLlama_{metric}'].mean()
            diff = gpt4_score - tiny_score
            
            html_content.append(f'<tr><td>{metric.replace("_", " ").title()}</td>'
                              f'<td>{gpt4_score:.3f}</td>'
                              f'<td>{tiny_score:.3f}</td>'
                              f'<td>{diff:+.3f}</td></tr>')
        
        # Add final scores
        gpt4_final = results_df['GPT4_Final_Score'].mean()
        tiny_final = results_df['TinyLlama_Final_Score'].mean()
        final_diff = gpt4_final - tiny_final
        
        html_content.append(f'<tr><th>Final Score</th>'
                          f'<th>{gpt4_final:.3f}</th>'
                          f'<th>{tiny_final:.3f}</th>'
                          f'<th>{final_diff:+.3f}</th></tr>')
        
        html_content.append('</table>')
        
        # Save the report
        with open(output_file, 'w') as f:
            f.write('\n'.join(html_content))
        
        print(f"\n📊 Performance analysis saved to {output_file}")

In [31]:
# Cell 3: Main Execution
def main():
    print("🚀 Starting Enhanced Math Problem Evaluation System")
    
    try:
        evaluator = ModelEvaluator(
            azure_endpoint=azure_endpoint,
            deployment_name=deployment_name,
            api_version=api_version,
            api_key=api_key
        )
        
        evaluator.evaluate_questions(
            input_csv='mmlu_math_questions.csv',
            output_csv=f'model_evaluation_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        )
        
    except Exception as e:
        print(f"\n❌ Error in main execution: {e}")
        raise

# Cell 4: Run the evaluation
if __name__ == "__main__":
    main()

🚀 Starting Enhanced Math Problem Evaluation System

🔄 Initializing Model Evaluator...


2024-12-01 12:03:48,077 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"


✅ Azure OpenAI client initialized successfully
✅ Ollama connection verified

📚 Loading questions from CSV...
✅ Loaded 60 questions for evaluation

Distribution of questions by subject:
Subject
Abstract Algebra           15
College Mathematics        15
Elementary Mathematics     15
High School Mathematics    15
Name: count, dtype: int64

📝 Question 1/60
Subject: Abstract Algebra
Question: Statement 1 | Every integral domain has a field of quotients. Statement 2 | A polynomial of degree n over a ring can have at most n zeros counting multiplicity.

🤖 GPT-4 [Question 1]


2024-12-01 12:03:55,054 - INFO - HTTP Request: POST https://access-01.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


⏱️ Time: 6.33s

🦙 TinyLlama [Question 1]


...

⏱️ Time: 13.77s

📊 Evaluating responses...

Detailed Scores for Question 59:

GPT-4:
correctness: 1.00
mathematical_reasoning: 0.35
solution_completeness: 0.47
explanation_quality: 0.35
coherence: 0.45
time_efficiency: 0.90
Final Score: 0.66

TinyLlama:
correctness: 1.00
mathematical_reasoning: 0.00
solution_completeness: 0.12
explanation_quality: 0.28
coherence: 0.10
time_efficiency: 0.70
Final Score: 0.49

Progress: 59/60 questions completed (98.3%)

📝 Question 60/60
Subject: High School Mathematics
Question: Let $h(4x-1) = 2x + 7$. For what value of $x$ is $h(x) = x$?

🤖 GPT-4 [Question 60]


2024-12-01 12:24:44,822 - INFO - HTTP Request: POST https://access-01.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"


⏱️ Time: 6.50s

🦙 TinyLlama [Question 60]


2024-12-01 12:24:59,150 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


⏱️ Time: 14.32s

📊 Evaluating responses...

Detailed Scores for Question 60:

GPT-4:
correctness: 1.00
mathematical_reasoning: 0.80
solution_completeness: 0.45
explanation_quality: 0.32
coherence: 0.40
time_efficiency: 0.90
Final Score: 0.84

TinyLlama:
correctness: 0.00
mathematical_reasoning: 0.10
solution_completeness: 0.04
explanation_quality: 0.00
coherence: 0.00
time_efficiency: 0.70
Final Score: 0.07

💾 Saving intermediate results...

Progress: 60/60 questions completed (100.0%)

📊 Final Statistics:
Total questions processed: 60
Total time taken: 21.17 minutes
Average time per question: 21.17 seconds

📊 Subject-wise Performance:

Abstract Algebra:
GPT-4 Average Score: 0.65
TinyLlama Average Score: 0.10

College Mathematics:
GPT-4 Average Score: 0.74
TinyLlama Average Score: 0.18

Elementary Mathematics:
GPT-4 Average Score: 0.59
TinyLlama Average Score: 0.19

High School Mathematics:
GPT-4 Average Score: 0.76
TinyLlama Average Score: 0.26

Overall Average Scores by Metric:

GPT-